In [5]:
#查看info,pkl中的内容 kitti_infos_train\n
import pickle
path = "/home/elodie/KITTI_DATASET/object/dbinfos_train.pkl"
with open(path, "rb") as f:
    kitti_infos = pickle.load(f)

In [12]:
print(len(kitti_infos))
print(len(kitti_infos["Car"]))
print(kitti_infos["Truck"][0])

8
14357
{'name': 'Truck', 'path': 'gt_database/16_Truck_1.bin', 'image_idx': 16, 'gt_idx': 1, 'box3d_lidar': array([26.70096943,  9.50276173,  0.23410452,  2.5999999 , 16.79000092,
        4.01999998,  1.76999998]), 'num_points_in_gt': 1967, 'difficulty': 0, 'group_id': 29}


In [ ]:
#         if res["type"] in ["NuScenesDataset", "LyftDataset"] and "gt_boxes" in info:

#             res["lidar"]["annotations"] = {
#                 "boxes": info["gt_boxes"].astype(np.float32),
#                 "names": info["gt_names"],
#                 "tokens": info["gt_boxes_token"],
#                 "velocities": info["gt_boxes_velocity"].astype(np.float32),
#             }

In [18]:
import numpy as np
from det3d.core import box_np_ops
info = kitti_infos[2]
if "annos" in info:
    annos = info["annos"]
    # we need other objects to avoid collision when sample
#     annos = kitti.remove_dontcare(annos) #elodie
    locs = annos["location"]
    dims = annos["dimensions"]
    rots = annos["rotation_y"]
    gt_names = annos["name"]
    gt_boxes = np.concatenate(
        [locs, dims, rots[..., np.newaxis]], axis=1
    ).astype(np.float32)
    calib = info["calib"]
    gt_boxes = box_np_ops.box_camera_to_lidar(
        gt_boxes, calib["R0_rect"], calib["Tr_velo_to_cam"]
    )

    # only center format is allowed. so we need to convert
    # kitti [0.5, 0.5, 0] center to [0.5, 0.5, 0.5]
    box_np_ops.change_box3d_center_(
        gt_boxes, [0.5, 0.5, 0], [0.5, 0.5, 0.5]
    )

    lidar_annotations = {
        "boxes": gt_boxes,
        "names": gt_names,
    }
    cam_annotations = {
        "boxes": annos["bbox"],
        "names": gt_names,
    }
print(info["point_cloud"]["velodyne_path"])
# print("gt_boxes",list(gt_boxes))
for gt_box in gt_boxes:
    print(list(gt_box),",")
print("gt_names",gt_names)
print("annos[bbox]",annos["bbox"])

training/velodyne/000007.bin
[25.29903469306752, 0.7089580661485912, -0.6884234406126626, 1.659999966621399, 3.200000047683716, 1.6100000143051147, -1.590000033378601] ,
[47.83820520269496, 7.4533924253269985, -0.6766321913155169, 1.5099999904632568, 3.700000047683716, 1.399999976158142, 1.5499999523162842] ,
[60.80635996946623, 4.733361323991922, -0.36983019503692494, 1.659999966621399, 4.050000190734863, 1.4600000381469727, 1.559999942779541] ,
[34.377720707453115, 12.651428590155888, -0.60237662969105, 0.5, 1.9500000476837158, 1.7200000286102295, 1.5399999618530273] ,
[-1010.3566390043871, 989.2525103619332, 999.4294837597301, -1.0, -1.0, -1.0, -10.0] ,
[-1010.3566390043871, 989.2525103619332, 999.4294837597301, -1.0, -1.0, -1.0, -10.0] ,
gt_names ['Car' 'Car' 'Car' 'Cyclist' 'DontCare' 'DontCare']
annos[bbox] [[564.62 174.59 616.43 224.74]
 [481.59 180.09 512.55 202.42]
 [542.05 175.55 565.27 193.79]
 [330.6  176.09 355.61 213.6 ]
 [753.33 164.32 798.   186.74]
 [738.5  171.32 753.

In [1]:
#创建新的数据集合

In [5]:
import numpy as np
import pickle
from det3d.core import box_np_ops
path = "/home/elodie/KITTI_DATASET/object/kitti_infos_val.pkl"
new_info_path = "/home/elodie/KITTI_DATASET_NEW/object/kitti_infos_val.pkl"
kitti_infos_new = []
R = np.array([[ 0, -1 ,0],
    [ 1,0,0],
    [ 0,0,1]])
# Car、 Van、 Truck、 Pedestrian、 Person_sitting、 Cyclist、 Tram、 Misc、 DontCare
car_list = ['Car', 'Van', 'Truck', 'Tram', ]
pedestrian_list = ['Pedestrian']
bicycle_list = ['Cyclist']
dont_list = ['Cyclist']
with open(path, "rb") as f:
    kitti_infos = pickle.load(f)

    for i in range(len(kitti_infos)):
        kitti_info = kitti_infos[i]
        ref_lidar_path = "/home/elodie/KITTI_DATASET_NEW/object/" + kitti_info["point_cloud"]["velodyne_path"] #绝对路径
        ref_cam_path = "/home/elodie/KITTI_DATASET_NEW/object/" + kitti_info["image"]["image_path"] #绝对路径

        cam_intrinsic_pre = kitti_info["calib"]["R0_rect"]
        cam_intrinsic_pre = np.delete(cam_intrinsic_pre, 3, axis=0)
        cam_intrinsic_pre = np.delete(cam_intrinsic_pre, 3, axis=1)
        ref_cam_intrinsic = cam_intrinsic_pre

        token = kitti_info["image"]["image_idx"]
        ref_from_car = kitti_info["calib"]["Tr_velo_to_cam"]
        car_from_global = kitti_info["calib"]["P0"]

        if "annos" in kitti_info:
            annos = kitti_info["annos"]
            # we need other objects to avoid collision when sample
            #     annos = kitti.remove_dontcare(annos) #elodie
            locs = annos["location"]
            dims = annos["dimensions"]
            rots = annos["rotation_y"]
            gt_names = annos["name"]
            gt_boxes = np.concatenate(
                [locs, dims, rots[..., np.newaxis]], axis=1
            ).astype(np.float32)
            calib = kitti_info["calib"]
            
            #box_camera_to_lidar [xyz_lidar, l, h, w, r] - > [xyz_lidar, w, l, h, r]
            gt_boxes = box_np_ops.box_camera_to_lidar(
                gt_boxes, calib["R0_rect"], calib["Tr_velo_to_cam"]
            )

            # only center format is allowed. so we need to convert
            # kitti [0.5, 0.5, 0] center to [0.5, 0.5, 0.5]
            box_np_ops.change_box3d_center_(
                gt_boxes, [0.5, 0.5, 0], [0.5, 0.5, 0.5]
            )
            #将gt_boxes 顺时针旋转90°
            gt_boxes_theta = np.dot(R, gt_boxes[:, :3].T).T
            gt_boxes = np.hstack((gt_boxes_theta, gt_boxes[:, 3:]))

            gt_boxes[:, 6] = gt_boxes[:, 6] - np.sign(gt_boxes[:, 6]) * np.pi / 2
            gt_boxes_velocity = np.zeros([gt_boxes.shape[0],3])

            gt_names_pre = annos["name"] #将不同类别合并
            for i in range(len(gt_names_pre)):
                if gt_names_pre[i] in car_list:
                    gt_names_pre[i] = 'car'
                elif gt_names_pre[i] in pedestrian_list:
                    gt_names_pre[i] = 'pedestrian'
                elif gt_names_pre[i] in bicycle_list:
                    gt_names_pre[i] = 'bicycle'
                else:
                    gt_names_pre[i] = 'dontcare'
            
        info = {
            "lidar_path": ref_lidar_path,
            "cam_front_path": ref_cam_path,
            "cam_intrinsic": ref_cam_intrinsic,
            "token": token,
            "sweeps": [],
            "ref_from_car": ref_from_car,
            "car_from_global": car_from_global,
            "timestamp": 0.0,
            "gt_boxes": gt_boxes,
            "gt_boxes_velocity": gt_boxes_velocity,
            "gt_names": gt_names_pre,
            "gt_boxes_token": np.zeros(gt_boxes.shape[0]),
            "data_type": "kitti"
        }
        kitti_infos_new.append(info)
        
        
        
with open(new_info_path, "wb") as f:
    pickle.dump(kitti_infos_new, f)

In [3]:
with open(new_info_path, "rb") as f:
    new_kitti_infos = pickle.load(f)
print(new_kitti_infos[0])
print(len(new_kitti_infos))

{'lidar_path': '/home/elodie/KITTI_DATASET_NEW/object/training/velodyne/000000.bin', 'cam_front_path': '/home/elodie/KITTI_DATASET_NEW/object/training/image_2/000000.png', 'cam_intrinsic': array([[ 0.9999128 ,  0.01009263, -0.00851193],
       [-0.01012729,  0.9999406 , -0.00403767],
       [ 0.00847068,  0.00412352,  0.9999556 ]]), 'token': 0, 'sweeps': [], 'ref_from_car': array([[ 0.00692796, -0.9999722 , -0.00275783, -0.02457729],
       [-0.00116298,  0.00274984, -0.9999955 , -0.06127237],
       [ 0.9999753 ,  0.00693114, -0.0011439 , -0.3321029 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), 'car_from_global': array([[707.0493,   0.    , 604.0814,   0.    ],
       [  0.    , 707.0493, 180.5066,   0.    ],
       [  0.    ,   0.    ,   1.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ,   1.    ]]), 'timestamp': 0.0, 'gt_boxes': array([[ 1.8559175 ,  8.73138176, -0.65469937,  0.47999999,  1.20000005,
         1.88999999, -1.56079633]]), 'gt_boxes_velocity':

In [7]:
import pickle
new_info_path = "/home/elodie/KITTI_DATASET_NEW/object/kitti_infos_val.pkl"
ll = ['car', 'pedestrian', 'bicycle']
with open(new_info_path, "rb") as f:
    new_infos = pickle.load(f)
print(new_infos[0])
for i in range(len(new_infos)):
#     print(new_infos[i]["gt_names"])
    gt_names = new_infos[i]["gt_names"]
    for n in gt_names:
        if n not in ll:
            print (i,n)
            print(gt_names)

{'lidar_path': '/home/elodie/KITTI_DATASET_NEW/object/training/velodyne/000001.bin', 'cam_front_path': '/home/elodie/KITTI_DATASET_NEW/object/training/image_2/000001.png', 'cam_intrinsic': array([[ 0.9999239 ,  0.00983776, -0.00744505],
       [-0.0098698 ,  0.9999421 , -0.00427846],
       [ 0.00740253,  0.00435161,  0.9999631 ]]), 'token': 1, 'sweeps': [], 'ref_from_car': array([[ 7.533745e-03, -9.999714e-01, -6.166020e-04, -4.069766e-03],
       [ 1.480249e-02,  7.280733e-04, -9.998902e-01, -7.631618e-02],
       [ 9.998621e-01,  7.523790e-03,  1.480755e-02, -2.717806e-01],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]]), 'car_from_global': array([[721.5377,   0.    , 609.5593,   0.    ],
       [  0.    , 721.5377, 172.854 ,   0.    ],
       [  0.    ,   0.    ,   1.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ,   1.    ]]), 'timestamp': 0.0, 'gt_boxes': array([[ 4.47564706e-01,  6.97247918e+01,  5.83652323e-01,
         2.63000011e+00,  1.23400002e+0

['car' 'car' 'dontcare' 'dontcare' 'dontcare' 'dontcare']
436 dontcare
['car' 'car' 'dontcare' 'dontcare' 'dontcare' 'dontcare']
436 dontcare
['car' 'car' 'dontcare' 'dontcare' 'dontcare' 'dontcare']
436 dontcare
['car' 'car' 'dontcare' 'dontcare' 'dontcare' 'dontcare']
437 dontcare
['car' 'car' 'car' 'dontcare']
438 dontcare
['car' 'car' 'dontcare' 'car' 'car' 'dontcare' 'dontcare' 'dontcare']
438 dontcare
['car' 'car' 'dontcare' 'car' 'car' 'dontcare' 'dontcare' 'dontcare']
438 dontcare
['car' 'car' 'dontcare' 'car' 'car' 'dontcare' 'dontcare' 'dontcare']
438 dontcare
['car' 'car' 'dontcare' 'car' 'car' 'dontcare' 'dontcare' 'dontcare']
439 dontcare
['car' 'car' 'car' 'dontcare' 'dontcare']
439 dontcare
['car' 'car' 'car' 'dontcare' 'dontcare']
441 dontcare
['car' 'car' 'car' 'car' 'dontcare' 'dontcare' 'dontcare' 'dontcare'
 'dontcare']
441 dontcare
['car' 'car' 'car' 'car' 'dontcare' 'dontcare' 'dontcare' 'dontcare'
 'dontcare']
441 dontcare
['car' 'car' 'car' 'car' 'dontcare' 'don

['car' 'car' 'car' 'car' 'bicycle' 'dontcare']
1000 dontcare
['car' 'car' 'dontcare']
1001 dontcare
['car' 'car' 'car' 'dontcare']
1002 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car'
 'car' 'dontcare' 'dontcare' 'dontcare']
1002 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car'
 'car' 'dontcare' 'dontcare' 'dontcare']
1002 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car'
 'car' 'dontcare' 'dontcare' 'dontcare']
1004 dontcare
['car' 'bicycle' 'car' 'car' 'dontcare' 'car' 'dontcare' 'dontcare']
1004 dontcare
['car' 'bicycle' 'car' 'car' 'dontcare' 'car' 'dontcare' 'dontcare']
1004 dontcare
['car' 'bicycle' 'car' 'car' 'dontcare' 'car' 'dontcare' 'dontcare']
1005 dontcare
['car' 'bicycle' 'car' 'dontcare' 'dontcare']
1005 dontcare
['car' 'bicycle' 'car' 'dontcare' 'dontcare']
1006 dontcare
['car' 'dontcare']
1007 dontcare
['car' 'car' 'car' 'car' 'dontcare' 'dontcare']
1007 dontcare
['car' 

['car' 'dontcare' 'dontcare' 'dontcare']
1498 dontcare
['car' 'dontcare' 'dontcare' 'dontcare']
1499 dontcare
['car' 'bicycle' 'pedestrian' 'pedestrian' 'car' 'dontcare' 'dontcare'
 'dontcare']
1499 dontcare
['car' 'bicycle' 'pedestrian' 'pedestrian' 'car' 'dontcare' 'dontcare'
 'dontcare']
1499 dontcare
['car' 'bicycle' 'pedestrian' 'pedestrian' 'car' 'dontcare' 'dontcare'
 'dontcare']
1500 dontcare
['car' 'car' 'bicycle' 'dontcare' 'dontcare' 'dontcare']
1500 dontcare
['car' 'car' 'bicycle' 'dontcare' 'dontcare' 'dontcare']
1500 dontcare
['car' 'car' 'bicycle' 'dontcare' 'dontcare' 'dontcare']
1501 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare']
1501 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare']
1502 dontcare
['car' 'car' 'car' 'dontcare']
1503 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car'
 'bicycle' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare']
1503 dontcare
['car' 'car' 

1976 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'dontcare' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
1976 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'dontcare' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
1976 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'dontcare' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
1976 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'dontcare' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
1976 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'dontcare' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
1977 dontcare
['car' 'car' 'dontcare' 'dontcare']
1977 dontcare
['car' 'car' 'dontcare' 'dontcare']
1978 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare']
1978 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare']
1979 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
1979 dontcare
[

2431 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
2431 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
2431 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
2432 dontcare
['car' 'dontcare' 'dontcare']
2432 dontcare
['car' 'dontcare' 'dontcare']
2433 dontcare
['pedestrian' 'pedestrian' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare']
2433 dontcare
['pedestrian' 'pedestrian' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare']
2434 dontcare
['car' 'car' 'car' 'car' 'dontcare']
2435 dontcare
['car' 'car' 'dontcare' 'dontcare' 'dontcare']
2435 dontcare
['car' 'car' 'dontcare' 'dontcare' 'dontcare']
2435 dontcare
['car' 'car' 'dontcare' 'dontcare' 'dontcare']
2436 dontcare
['car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare' 'dontcare']
2436 dontcare
['car' 'car' 'car' 'car

2945 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'bicycle' 'bicycle' 'pedestrian'
 'dontcare' 'dontcare' 'dontcare' 'dontcare']
2945 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'bicycle' 'bicycle' 'pedestrian'
 'dontcare' 'dontcare' 'dontcare' 'dontcare']
2946 dontcare
['car' 'car' 'car' 'car' 'car' 'dontcare']
2948 dontcare
['car' 'car' 'car' 'car' 'car' 'dontcare']
2949 dontcare
['car' 'bicycle' 'car' 'car' 'bicycle' 'dontcare' 'dontcare' 'dontcare']
2949 dontcare
['car' 'bicycle' 'car' 'car' 'bicycle' 'dontcare' 'dontcare' 'dontcare']
2949 dontcare
['car' 'bicycle' 'car' 'car' 'bicycle' 'dontcare' 'dontcare' 'dontcare']
2950 dontcare
['car' 'car' 'car' 'dontcare']
2951 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare']
2951 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare']
2952 dontcare
['car' 'car' 'car' 'car' 'car' 'pedestrian' 'dontcare' 'dontcare'
 'dontcare' 'dontcare' 'dontcare']
2952 dontcare

3467 dontcare
['car' 'car' 'car' 'dontcare' 'car' 'dontcare' 'dontcare']
3467 dontcare
['car' 'car' 'car' 'dontcare' 'car' 'dontcare' 'dontcare']
3469 dontcare
['car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare']
3469 dontcare
['car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare']
3470 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare']
3470 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'car' 'dontcare'
 'dontcare']
3471 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'pedestrian' 'pedestrian'
 'pedestrian' 'pedestrian' 'pedestrian' 'car' 'dontcare' 'dontcare']
3471 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'pedestrian' 'pedestrian'
 'pedestrian' 'pedestrian' 'pedestrian' 'car' 'dontcare' 'dontcare']
3474 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare']
3474 dontcare
['car' 'car' 'car' 'car' 'car' 'car' 'dontcare' 'dontcare']
3475 dontcare
['pedestrian' 'pedestrian' 'pedestrian' 'pedestrian' 'pedest

In [ ]:
#将kitti的点云顺时针旋转90度后，生成新的数据集合

In [1]:
import os
import numpy as np

R = np.array([[ 0, -1 ,0],
    [ 1,0,0],
    [ 0,0,1]])

velodyne_path = "/home/elodie/KITTI_DATASET/object/training/velodyne_reduced"
new_velodyne_path = "/home/elodie/KITTI_DATASET_NEW/object/training/velodyne_reduced"

velodyne_path_list = []
new_velodyne_path_list = []

for root,dirs,files in os.walk(velodyne_path):
    for file in files:
        #获取文件路径
        #print(os.path.join(root,file))
        velodyne_path_list.append(os.path.join(root,file))
        new_velodyne_path_list.append(os.path.join(new_velodyne_path,file))
        
for i in range(len(velodyne_path_list)):
    print(i)
    if os.path.exists(new_velodyne_path_list[i]):
        continue
    points = np.fromfile(velodyne_path_list[i], dtype=np.float32, count=-1).reshape([-1, 4])
    
    points_xy = np.hstack((points[:,:2],np.ones((points.shape[0],1))))
    points_theta = np.dot(R,points_xy.T).T
    points_theta = np.hstack((points_theta[:,:2],points[:,2:]))
    
    points_theta.tofile(new_velodyne_path_list[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064


3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708


5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355


6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995


In [4]:
import os
new_velodyne_path = "/home/elodie/KITTI_DATASET/object/training/velodyne"
new_files = []
for root,dirs,files in os.walk(new_velodyne_path):
    for file in files:
        #获取文件路径
        #print(os.path.join(root,file))
        new_files.append(os.path.join(root,file))
print(len(new_files))

7481


In [10]:
points = np.fromfile(new_files[1], dtype=np.float32, count=-1).reshape([-1, 4])
print(points)
points_xy = np.hstack((points[:,:2],np.ones((points.shape[0],1))))
points_theta = np.dot(R,points_xy.T).T
points_theta = np.hstack((points_theta[:,:2],points[:,2:]))
print("points_theta,",points_theta)

[[73.965 12.714  2.747  0.   ]
 [66.849 11.922  2.505  0.   ]
 [73.997 13.439  2.752  0.   ]
 ...
 [ 3.786 -1.426 -1.77   0.45 ]
 [ 3.724 -1.389 -1.737  0.18 ]
 [ 3.81  -1.408 -1.778  0.   ]]
points_theta, [[-12.71399975  73.96499634   2.74699998   0.        ]
 [-11.92199993  66.84899902   2.50500011   0.        ]
 [-13.43900013  73.99700165   2.75200009   0.        ]
 ...
 [  1.426        3.78600001  -1.76999998   0.44999999]
 [  1.38900006   3.72399998  -1.73699999   0.18000001]
 [  1.40799999   3.80999994  -1.778        0.        ]]


In [1]:
import os
new_velodyne_path = "/home/elodie/KITTI_DATASET_NEW/object/training/velodyne"
new_files2 = []
for root,dirs,files in os.walk(new_velodyne_path):
    for file in files:
        #获取文件路径
        #print(os.path.join(root,file))
        new_files2.append(os.path.join(root,file))
print(len(new_files2))

7481


In [3]:
import numpy as np
points2 = np.fromfile(new_files2[1],dtype=np.float32).reshape([-1, 4])
print(points2)

[[ 3.6893488e+19 -2.6473124e+00 -0.0000000e+00  3.2889256e+00]
 [ 1.0842022e-19  2.0933750e+00  0.0000000e+00  0.0000000e+00]
 [ 3.6893488e+19 -2.6225624e+00  0.0000000e+00  3.2611289e+00]
 ...
 [-0.0000000e+00 -1.9671249e+00 -0.0000000e+00  1.5549999e+00]
 [ 0.0000000e+00  1.9260000e+00  2.0000000e+00  2.2262499e+00]
 [ 1.0842022e-19 -1.9722500e+00  0.0000000e+00  0.0000000e+00]]
